In [ ]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import outils
%matplotlib inline

## Réseau de Convolution avec Tensorflow

https://www.tensorflow.org/get_started/mnist/pros

- on reprends le cas des chiffres manuscrits de MNIST
- mais on va utiliser des convolutions
- on se limite à un réseau plus simple que le tutorial

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# cela télécharge MNIST au besoin

In [ ]:
NC = 10  # nombre de classes
D = 784  # dimension de l'espace d'entrée (images de 28x28)

In [ ]:
# entree
x = tf.placeholder(tf.float32, [None, D])
x_image = tf.reshape(x, [-1,28,28,1])     # remettre les images en 2D (batch, x, y, features)

def convrelu(X, fr, to):
    w = tf.Variable(tf.truncated_normal([3,3, fr, to], stddev=0.05))
    b = tf.Variable(tf.truncated_normal([to], stddev=0.05))
    c = tf.nn.conv2d(X, w, [1, 1,1, 1], "VALID")
    o = tf.nn.relu(tf.nn.max_pool(c + b, [1,2,2,1], [1,2,2,1], "VALID"))
    return w,b,o

def fc(X, to, relu=True):
    fr = int(X.shape[1])
    w = tf.Variable(tf.truncated_normal([fr, to], stddev=0.05))
    b = tf.Variable(tf.zeros([to]))
    o = tf.matmul(X, w) + b
    if relu:
        o = tf.nn.relu(o)
    return w,b,o


# model à deux couches
NF1 = 16
NF2 = 128

_,_, o1 = convrelu(x_image, 1, NF1)
_,_, o2 = convrelu(o1, NF1, NF2)
_,_, o3 = convrelu(o2, NF2, NF2)

# converge avec un tout-connecté
o = o3
NPRE = int(np.prod(o.shape[1:]))
print(o.shape, NPRE)

_,_,fc1 = fc(tf.reshape(o, [-1, NPRE]), 100)
_,_,fc2 = fc(fc1, NC, relu=False)

y = fc2

In [ ]:
# étiquette cible
y_ = tf.placeholder(tf.float32, [None, NC])

# fonction de perte (sorte de maximisation de la probabilité prédite par le réseau)
cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

# optimiseur
train_step = tf.train.GradientDescentOptimizer(0.25).minimize(cross_entropy)

In [ ]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    # Entrainement
    for iteration in range(1000):
        # traiter les exemples 100 par 100 (« Stochastic Gradient Descent »)
        batch_xs, batch_ys = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

    # Tester le model appris
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    print("Precision entrainement :", sess.run(accuracy, feed_dict={x: mnist.train.images, y_: mnist.train.labels}))
    print("Precision test :", sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))


### À Vous
- reprendre ou modifier le code ci dessus
- faire que la couche cachée aie 30 unités (neurones)
- calculer l'accuracy
- ajouter une activation/non-linéarité `tf.nn.relu` à la sortie de la couche cachée
- calculer l'accuracy
- ajouter une couche cachée avec 25 unités et une ReLU
- calculer l'accuracy